In [21]:
from openai import OpenAI
import json

client = OpenAI(api_key="sk-3fc5030e1f144b30990ef0c40940c66e", base_url="https://api.deepseek.com")

In [22]:
def build_messages(text):
    return [
        {
            "role": "system",
            "content": (
                "You are an expert in English grammar and writing. "
                "You are helping Chinese students prepare for the CET-4 and CET-6 exams. "
                "Your task is to analyze a complex English passage and extract:\n"
                "1) paragraph structure,\n"
                "2) beautiful expressions,\n"
                "3) grammar highlights,\n"
                "4) writing techniques.\n\n"
                "Return your response in **valid JSON format only**, no explanations, no extra comments. "
                "The structure should look like this:\n\n"
                "{\n"
                '  "structure": ["主题句：...", "扩展说明：..."],\n'
                '  "expressions": [\n'
                '    { "phrase": "quiet joys", "meaning": "静谧的喜悦", "comment": "quiet的妙用" }\n'
                "  ],\n"
                '  "grammar": ["被动语态...", "现在分词..."],\n'
                '  "tips": ["双重好处表达法...", "场景描写技巧..."]\n'
                "}\n\n"
                "Respond in Chinese."
            )
        },
        {
            "role": "user",
            "content": f"请分析以下英文段落：\n\n\"{text}\""
        }
    ]

In [23]:
def analyze(text):
    
    messages = build_messages(text)
    response = client.chat.completions.create(
        # response_format={"type": "json_object"},
        model="deepseek-chat",
        messages=messages,
        max_tokens=500,
    )
    return response.choices[0].message.content

In [24]:

test_sentence = "Despite being warned repeatedly, he remained indifferent to the consequences, which eventually led to his downfall."
result = analyze(test_sentence)


In [26]:
#函数提纯json内容
import re

def extract_json(text):
    # 匹配 ```json\n{...}\n``` 结构或直接 { 开头
    match = re.search(r"```json\s*({.*?})\s*```", text, re.DOTALL)
    if match:
        return match.group(1)
    elif text.strip().startswith("{"):
        return text.strip()
    else:
        return None


In [29]:
json_text = extract_json(result)
print(json_text)

<class 'str'>
{
  "structure": ["主题句：Despite being warned repeatedly, he remained indifferent to the consequences", "扩展说明：which eventually led to his downfall"],
  "expressions": [
    { "phrase": "remained indifferent", "meaning": "保持冷漠", "comment": "remain的持续状态表达" },
    { "phrase": "eventually led to", "meaning": "最终导致", "comment": "因果关系的递进表达" }
  ],
  "grammar": ["被动语态：being warned", "定语从句：which eventually led to his downfall"],
  "tips": ["警告与结果的对比手法", "后果递进描写技巧"]
}
